In [ ]:
import sys
from glob import glob
from os import path as osp
from datetime import datetime
from tqdm import tqdm
from skimage import io, transform
from matplotlib import pyplot as plt
import numpy as np
from math import sqrt
from statistics import mean
# torch imports
import torch
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader

# root path of project
from os import path as osp
import sys

# get root directory
import re
reg = '^.*/AquaPose'
project_root = re.findall(reg, osp.dirname(osp.abspath(sys.argv[0])))[0]
sys.path.append(project_root)

from lib.dataset.PoseDataset import PoseDataset

from lib.models.keypoint_rcnn import get_resnet50_pretrained_model

# utils
from lib.utils.slack_notifications import slack_message
from lib.utils.select_gpu import select_best_gpu
from lib.utils.rmsd import kabsch_rmsd, kabsch_rotate, kabsch_weighted_rmsd, centroid, centroid_weighted, rmsd, rmsd_weighted, kabsch

# references import
# source: https://github.com/pytorch/vision/tree/master/references/detection
from references.engine import train_one_epoch, evaluate
from references.utils import collate_fn

from references.transforms import RandomHorizontalFlip

from lib.matching.matching import *
from lib.utils.visual_utils import *

In [ ]:
# load dataset to get a set of poses to match to
ref_dataset = PoseDataset([osp.join(project_root,'data/vzf/freestyle/freestyle_1'), osp.join(project_root,'data/vzf/freestyle/freestyle_2'), osp.join(project_root,'data/vzf/freestyle/freestyle_3'), osp.join(project_root,'data/vzf/freestyle/freestyle_4')], train=False)

inference_dataset = PoseDataset([osp.join(project_root,'data/vzf/freestyle/freestyle_5')], train=False)

In [ ]:
weight_dir = osp.join(project_root, 'weights')
weight_files = glob(osp.join(weight_dir,'*'))
model = get_resnet50_pretrained_model()
#print(weight_files)
model.load_state_dict(torch.load(weight_files[1], map_location=torch.device('cpu')))

In [ ]:
#anchor_ids = [40,43,45,47,48,50,52,54,57,58]
anchor_ids = [40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]
anchor_dataset = torch.utils.data.Subset(ref_dataset, anchor_ids)

In [ ]:
model.eval()
img_ids_list, obs_lik_list, observations_list, hidden_states_list, flipped_list= get_observation_likelihood_and_hidden_state(model, inference_dataset, anchor_dataset, max_stride=3)

In [ ]:
print(img_ids_list)

In [ ]:
def build_transmat(num_states, probs=[.25,.25,.2,.15,.1,.5]):
    transmat = np.zeros((num_states,num_states))
    for i in range (0,num_states):
        for offset, prob in enumerate(probs):
            transmat[i,(i+offset)%num_states] = prob
    return transmat

# transmat = np.array(
#     [
#         [.5,.4,.1,0,0,0,0,0,0,0],
#         [0,.5,.4,.1,0,0,0,0,0,0],
#         [0,0,.5,.4,.1,0,0,0,0,0],
#         [0,0,0,.5,.4,.1,0,0,0,0],
#         [0,0,0,0,.5,.4,.1,0,0,0],
#         [0,0,0,0,0,.5,.4,.1,0,0],
#         [0,0,0,0,0,0,.5,.4,.1,0],
#         [0,0,0,0,0,0,0,.5,.4,.1],
#         [.1,0,0,0,0,0,0,0,.5,.4],
#         [.4,.1,0,0,0,0,0,0,0,.5]
#     ]
#     )

# transmat = np.array(
#     [
#         [.4,.5,.1,0,0,0,0,0,0,0],
#         [0,.4,.5,.1,0,0,0,0,0,0],
#         [0,0,.4,.5,.1,0,0,0,0,0],
#         [0,0,0,.4,.5,.1,0,0,0,0],
#         [0,0,0,0,.4,.5,.1,0,0,0],
#         [0,0,0,0,0,.4,.5,.1,0,0],
#         [0,0,0,0,0,0,.4,.5,.1,0],
#         [0,0,0,0,0,0,0,.4,.5,.1],
#         [.1,0,0,0,0,0,0,0,.4,.5],
#         [.5,.1,0,0,0,0,0,0,0,.4]
#     ]
#     )

transmat = build_transmat(len(anchor_ids))


In [ ]:
for img_ids, obs_lik, obs, hid, flipped_mat in zip(img_ids_list, obs_lik_list, observations_list, hidden_states_list, flipped_list):
    print('hidden {}'.format(hid))
    print('obs: {}'.format(obs))
    mls = viterbi_path(np.array([0.1]*len(anchor_ids)), transmat, obs_lik)
    print('viterbi: {}'.format(mls))
    print('obslik: {}'.format(obs_lik))
    print('flipped_mat {}'.format(flipped_mat))

    for obs_id, (img, anchor) in tqdm(enumerate(zip(img_ids, mls))):
        inf_img, _ = inference_dataset[img]
        warped_anchor = warp_anchor_on_pred(model, inf_img, anchor_dataset, anchor, False)
        plot_image_with_kps_skeleton(inf_img, [warped_anchor])

    # 

## warp viterbi corrected anchor pose on image